In [1]:
import pandas as pd
import pickle

# Linear models
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier

# Naive Bayes
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

# Support Vector Machines
from sklearn.svm import LinearSVC

# Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

# Tree-based models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Pipelines and preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

# Model selection
from sklearn.model_selection import StratifiedKFold, cross_validate

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)

# External models
from xgboost import XGBClassifier


In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Load models

In [3]:
with open("../models/ml_models_classyfire.pkl", "rb") as f:
    models = pickle.load(f)

In [4]:
models

{'Logistic Regression': LogisticRegression(solver='liblinear'),
 'BernoulliNB': BernoulliNB(),
 'Multinomial Naive Bayes': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('multinomialnb', MultinomialNB())]),
 'Linear SVM': LinearSVC(),
 'k-NN': KNeighborsClassifier(),
 'Random Forest': RandomForestClassifier(),
 'LDA': LinearDiscriminantAnalysis(),
 'SGD Classifier': SGDClassifier(),
 'Lasso Regression': LogisticRegression(penalty='l1', solver='saga'),
 'Ridge Regression': RidgeClassifier(),
 'Elastic Net': LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 'Decision Tree': DecisionTreeClassifier(),
 'Gradient Boosting': GradientBoostingClassifier()}

# Load data for calibration LinearSVC, SGDClassifier, RidgeClassifier

In [5]:
with open("../data/antidiabetic/antidiabetic_classyfire_chemont_taxonomy.pkl", "rb") as f:
    antidiabetic_taxonomy_descriptors = pickle.load(f)

In [6]:
with open("../data/non_antidiabetic/non_antidiabetic_classyfire_chemont_taxonomy.pkl", "rb") as f:
    non_antidiabetic_taxonomy_descriptors = pickle.load(f)

In [7]:
df_a = pd.DataFrame.from_dict(antidiabetic_taxonomy_descriptors).T
df_a = df_a.applymap(lambda x: 1 if isinstance(x, str) else -1)
df_N = pd.DataFrame.from_dict(non_antidiabetic_taxonomy_descriptors).T
df_N = df_N.applymap(lambda x: 1 if isinstance(x, str) else -1)

/tmp/ipykernel_1263/2257884334.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_a = df_a.applymap(lambda x: 1 if isinstance(x, str) else -1)
/tmp/ipykernel_1263/2257884334.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_N = df_N.applymap(lambda x: 1 if isinstance(x, str) else -1)


In [8]:
df_a['y'] = 1
df_N['y'] = -1

df = pd.concat([df_a, df_N], ignore_index=True)

In [9]:
X = df.drop(columns="y")
y = df["y"]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Get classyfire chemont taxonomy

In [11]:
response_text = '''{"id":12133074,"label":"MyQueryLabel","classification_status":"Done","number_of_elements":1,"number_of_pages":1,"invalid_entities":[],"entities":[{"identifier":"Q12133074-1","smiles":"CC@HC1=CC=CC=C1","inchikey":"InChIKey=WAPNOHKVXSQRPX-ZETCQYMHSA-N","kingdom":{"name":"Organic compounds","description":"Compounds that contain at least one carbon atom, excluding isocyanide/cyanide and their non-hydrocarbyl derivatives, thiophosgene, carbon diselenide, carbon monosulfide, carbon disulfide, carbon subsulfide, carbon monoxide, carbon dioxide, Carbon suboxide, and dicarbon monoxide.","chemont_id":"CHEMONTID:0000000","url":"http://classyfire.wishartlab.com/tax_nodes/C0000000"},"superclass":{"name":"Benzenoids","description":"Aromatic compounds containing one or more benzene rings.","chemont_id":"CHEMONTID:0002448","url":"http://classyfire.wishartlab.com/tax_nodes/C0002448"},"class":{"name":"Benzene and substituted derivatives","description":"Aromatic compounds containing one monocyclic ring system consisting of benzene.","chemont_id":"CHEMONTID:0002279","url":"http://classyfire.wishartlab.com/tax_nodes/C0002279"},"subclass":null,"intermediate_nodes":[],"direct_parent":{"name":"Benzene and substituted derivatives","description":"Aromatic compounds containing one monocyclic ring system consisting of benzene.","chemont_id":"CHEMONTID:0002279","url":"http://classyfire.wishartlab.com/tax_nodes/C0002279"},"alternative_parents":[{"name":"Secondary alcohols","description":"Compounds containing a secondary alcohol functional group, with the general structure HOC(R)(R') (R,R'=alkyl, aryl).","chemont_id":"CHEMONTID:0001661","url":"http://classyfire.wishartlab.com/tax_nodes/C0001661"},{"name":"Hydrocarbon derivatives","description":"Derivatives of hydrocarbons obtained by substituting one or more carbon atoms by an heteroatom. They contain at least one carbon atom and heteroatom.","chemont_id":"CHEMONTID:0004150","url":"http://classyfire.wishartlab.com/tax_nodes/C0004150"},{"name":"Aromatic alcohols","description":"Compounds containing an alcohol group attached to an aromatic carbon.","chemont_id":"CHEMONTID:0003073","url":"http://classyfire.wishartlab.com/tax_nodes/C0003073"}],"molecular_framework":"Aromatic homomonocyclic compounds","substituents":["Monocyclic benzene moiety","Secondary alcohol","Organic oxygen compound","Hydrocarbon derivative","Aromatic alcohol","Organooxygen compound","Alcohol","Aromatic homomonocyclic compound"],"description":"This compound belongs to the class of organic compounds known as benzene and substituted derivatives. These are aromatic compounds containing one monocyclic ring system consisting of benzene.","external_descriptors":[{"source":"CHEBI","source_id":"CHEBI:16346","annotations":["1-phenylethanol"]}],"ancestors":["Alcohols and polyols","Aromatic alcohols","Benzene and substituted derivatives","Benzenoids","Chemical entities","Hydrocarbon derivatives","Organic compounds","Organic oxygen compounds","Organooxygen compounds","Secondary alcohols"],"predicted_chebi_terms":["secondary alcohol (CHEBI:35681)","organic molecule (CHEBI:72695)","aromatic alcohol (CHEBI:33854)","benzenes (CHEBI:22712)","chemical entity (CHEBI:24431)","oxygen molecular entity (CHEBI:25806)","organic molecular entity (CHEBI:50860)","organooxygen compound (CHEBI:36963)","polyol (CHEBI:26191)","organic hydroxy compound (CHEBI:33822)","alcohol (CHEBI:30879)","benzenoid aromatic compound (CHEBI:33836)"],"predicted_lipidmaps_terms":[],"classification_version":"2.1"}]}'''

In [12]:
import json
from typing import Dict, Any, List

def parse_classyfire_ids(response_text: str) -> Dict[str, str]:
    """
    Extracts identifiers and names from the response of the ClassyFire API.
    
    Args:
        response_text: A JSON string from the ClassyFire API response
    
    Returns:
        Dict[str, str]: A dictionary where keys are identifiers and values are names
        
    Raises:
        json.JSONDecodeError: if the JSON is malformed
        KeyError: if required fields are missing in the data
    """

    result: Dict[str, str] = {}
    
    try:
        data = json.loads(response_text)
        
        # Get the first entity from the list
        if not data.get('entities') or not data['entities'][0]:
            return result
            
        entity = data['entities'][0]
        
        # Processing predicted_chebi_terms
        chebi_terms: List[str] = entity.get('predicted_chebi_terms', [])
        for term in chebi_terms:
            # Extract CHEBI ID and name from a string in the format "name (CHEBI:id)"
            if '(' in term and ')' in term:
                name, chebi_id = term.split(' (')
                chebi_id = chebi_id.rstrip(')')
                result[chebi_id] = name
        
        # Processing all nodes with CHEMONTID
        nodes_to_check = [
            entity.get('kingdom', {}),
            entity.get('superclass', {}),
            entity.get('class', {}),
            entity.get('subclass', {}),
            entity.get('direct_parent', {})
        ]
        nodes_to_check.extend(entity.get('alternative_parents', []))
        nodes_to_check.extend(entity.get('intermediate_nodes', []))
        
        for node in nodes_to_check:
            if node and 'chemont_id' in node and 'name' in node:
                result[node['chemont_id']] = node['name']
                
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing data: {e}")
        
    return result

In [13]:
taxonomy_descriptors = parse_classyfire_ids(response_text)
#df_taxonomy_descriptors = pd.DataFrame.from_dict(taxonomy_descriptors).T
df_taxonomy_descriptors = pd.DataFrame.from_dict(taxonomy_descriptors, orient='index')
df_taxonomy_descriptors = df_taxonomy_descriptors.applymap(lambda x: 1 if isinstance(x, str) else -1).T
df_taxonomy_descriptors

/tmp/ipykernel_1263/2237630750.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_taxonomy_descriptors = df_taxonomy_descriptors.applymap(lambda x: 1 if isinstance(x, str) else -1).T


,CHEBI:35681,CHEBI:72695,CHEBI:33854,CHEBI:22712,CHEBI:24431,CHEBI:25806,CHEBI:50860,CHEBI:36963,CHEBI:26191,CHEBI:33822,CHEBI:30879,CHEBI:33836,CHEMONTID:0000000,CHEMONTID:0002448,CHEMONTID:0002279,CHEMONTID:0001661,CHEMONTID:0004150,CHEMONTID:0003073
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# Predict antidiabetic effect

In [14]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.impute import SimpleImputer
import numpy as np
from tqdm import tqdm

def prepare_input(df, expected_features, imputer=None, fit_imputer=False):
    """
    Prepare input data by ensuring all expected features are present
    and handling missing values.
    """
    df_copy = df.copy()
    
    # Find missing columns
    missing_cols = [col for col in expected_features if col not in df_copy.columns]
    
    # Add missing columns efficiently using pd.concat
    if missing_cols:
        missing_df = pd.DataFrame(
            np.full((len(df_copy), len(missing_cols)), np.nan),
            columns=missing_cols,
            index=df_copy.index
        )
        df_copy = pd.concat([df_copy, missing_df], axis=1)
    
    # Select only the expected features
    df_prepared = df_copy[expected_features]
    
    # Handle missing values
    if imputer is not None:
        if fit_imputer:
            # Fit and transform (for training data)
            df_prepared = imputer.fit_transform(df_prepared)
        else:
            # Only transform (for test/new data)
            df_prepared = imputer.transform(df_prepared)
        
        # Convert back to DataFrame to maintain column names
        df_prepared = pd.DataFrame(df_prepared, columns=expected_features, index=df_copy.index)
    
    return df_prepared

# Store feature names and imputers for each model
saved_feature_names = {}
saved_imputers = {}

print("Preparing imputers for models...")
for model_name, model in tqdm(models.items(), desc="Setting up imputers"):
    saved_feature_names[model_name] = X_train.columns.tolist()
    
    # Create and fit imputer for models that can't handle NaN
    # Most sklearn models can't handle NaN, only some tree-based and specific models can
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
    from sklearn.tree import DecisionTreeClassifier
    
    # Models that CAN handle NaN natively (very few)
    nan_tolerant_models = (HistGradientBoostingClassifier,)
    
    if isinstance(model, nan_tolerant_models):
        saved_imputers[model_name] = None
    else:
        # All other models need imputation
        imputer = SimpleImputer(strategy='median')  # or 'mean', 'most_frequent'
        saved_imputers[model_name] = imputer

# Make predictions
predictions = {}
print("\nMaking predictions with models...")
for model_name, model in tqdm(models.items(), desc="Processing models"):
    # Get expected features for this model
    expected_features = model.feature_names_in_
    imputer = saved_imputers[model_name]
    
    print(f"  Processing {model_name}...")
    
    # Prepare test data
    print(f"    Preparing test data...")
    X_test_prepared = prepare_input(X_test.copy(), expected_features, imputer, fit_imputer=True)
    
    # Calibrate if needed (only for specific models that don't have predict_proba)
    if isinstance(model, (LinearSVC, SGDClassifier, RidgeClassifier)):
        print(f"    Calibrating {model_name}...")
        calibrated = CalibratedClassifierCV(model, cv="prefit")
        calibrated.fit(X_test_prepared, y_test)
        
        print(f"    Making predictions with calibrated {model_name}...")
        # Prepare input data for prediction
        df_input = prepare_input(df_taxonomy_descriptors.copy(), expected_features, imputer, fit_imputer=False)
        proba = calibrated.predict_proba(df_input)
    else:
        print(f"    Making predictions with {model_name}...")
        # For all other models, prepare input with imputation if needed
        df_input = prepare_input(df_taxonomy_descriptors.copy(), expected_features, imputer, fit_imputer=False)
        proba = model.predict_proba(df_input)
    
    predictions[model_name] = proba[:, 1]
    print(f"    ✓ {model_name} completed")

print("Predictions completed successfully!")
print(f"Models processed: {list(predictions.keys())}")
print(f"Predictions shape for each model: {[pred.shape for pred in predictions.values()]}")

Preparing imputers for models...


Setting up imputers: 100%|███████████████████| 13/13 [00:00<00:00, 28281.10it/s]



Making predictions with models...


Processing models:   8%|█▉                       | 1/13 [00:00<00:02,  5.18it/s]

  Processing Logistic Regression...
    Preparing test data...
    Making predictions with Logistic Regression...
    ✓ Logistic Regression completed
  Processing BernoulliNB...
    Preparing test data...


...

    Calibrating Ridge Regression...
    Making predictions with calibrated Ridge Regression...
    ✓ Ridge Regression completed
  Processing Elastic Net...
    Preparing test data...


Processing models:  85%|████████████████████▎   | 11/13 [00:02<00:00,  4.80it/s]

    Making predictions with Elastic Net...
    ✓ Elastic Net completed
  Processing Decision Tree...
    Preparing test data...


Processing models: 100%|████████████████████████| 13/13 [00:02<00:00,  4.52it/s]

    Making predictions with Decision Tree...
    ✓ Decision Tree completed
  Processing Gradient Boosting...
    Preparing test data...
    Making predictions with Gradient Boosting...
    ✓ Gradient Boosting completed
Predictions completed successfully!
Models processed: ['Logistic Regression', 'BernoulliNB', 'Multinomial Naive Bayes', 'Linear SVM', 'k-NN', 'Random Forest', 'LDA', 'SGD Classifier', 'Lasso Regression', 'Ridge Regression', 'Elastic Net', 'Decision Tree', 'Gradient Boosting']
Predictions shape for each model: [(1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,)]


In [15]:
predict_proba = pd.DataFrame({k: pd.Series(v) for k, v in predictions.items()})
predict_proba

,Logistic Regression,BernoulliNB,Multinomial Naive Bayes,Linear SVM,k-NN,Random Forest,LDA,SGD Classifier,Lasso Regression,Ridge Regression,Elastic Net,Decision Tree,Gradient Boosting
0,0.001508,0.994312,0.003167,0.000161,0.6,0.0,9.464445e-214,0.001871,0.014785,0.000277,0.009341,0.0,0.019485
